In [1]:
import multiprocessing  
import sys
sys.path.insert(0, '..')
import time
import matplotlib.pyplot as plt
%matplotlib inline  

import numpy as np

from Configuration import Configuration
from MotorUnit import MotorUnit
from NeuralTract import NeuralTract
from SynapsesFactory import SynapsesFactory
from jointAnkleForceTask import jointAnkleForceTask

In [2]:
def func(n):
    for i in range(10000):
        for j in range(10000):
            s=j*i
    print(n)
 

In [3]:
pool = multiprocessing.Pool(processes=4)
pool.map(func, range(10))
pool.close()
pool.join()   
print('done')

3
2
0
1
5
4
7
6
8
9
done


In [4]:
from math import sqrt
from joblib import Parallel, delayed
Parallel(n_jobs=2)(delayed(sqrt)(i ** 2) for i in range(1000))



[0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 88.0,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0,
 95.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 104.0,
 105.0,
 106.0,
 107.0,
 108.0,
 109.0,
 110.0,
 111.0,
 112.0,
 113.0,
 114.0,
 115.0,
 116.0,
 117.0,
 118.0,
 119.0,
 120.0,
 121.0,
 122.0,
 123.0,
 124.0,
 125.0,
 126.0,
 127.0,
 128.0,
 129.0,
 130.0,
 131.0,
 132.0,
 133.0,
 134.0,
 135.0,
 136.0,
 137.0,
 138.0

In [5]:
conf = Configuration('confMuscleHillExample.rmto')

In [6]:
unit = dict()

for i in xrange(0, 100): 
            if i < 100:
                unit[i] = MotorUnit(conf, 'SOL', i, 'S', 18, conf.skinThickness_mm)
            elif i < 100 + 0:
                unit[i] = MotorUnit(conf, 'SOL', i, 'FR', 18, conf.skinThickness_mm)
            else:
                unit[i] = MotorUnit(conf, 'SOL', i, 'FF', 18, conf.skinThickness_mm)


In [7]:
t = np.arange(0.0, conf.simDuration_ms, conf.timeStep_ms)
print conf.simDuration_ms

100.0


In [8]:
tic = time.clock()
for j in xrange(0,len(t)-1): 
    [unit[i].atualizeMotorUnit(t[j]) for i in xrange(100)]
toc = time.clock()
print str(toc - tic) + ' seconds'

19.861099 seconds


In [9]:
unit1 = dict()
for i in xrange(0, 100): 
            if i < 100:
                unit1[i] = MotorUnit(conf, 'SOL', i, 'S', 18, conf.skinThickness_mm)
            elif i < 100 + 0:
                unit1[i] = MotorUnit(conf, 'SOL', i, 'FR', 18, conf.skinThickness_mm)
            else:
                unit1[i] = MotorUnit(conf, 'SOL', i, 'FF', 18, conf.skinThickness_mm)


In [10]:
from joblib import Parallel, delayed
import dill
tic = time.clock()
for j in xrange(0,len(t)-1):
    instant= t[j]
    Parallel(n_jobs=2)(delayed(unit1[i].atualizeMotorUnit)(instant) for i in xrange(100))
toc = time.clock()
print str(toc - tic) + ' seconds'

TypeError: can't pickle instancemethod objects